In [1]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np

import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [3]:
def get_process(vi,pi,vf,j):
    '''
    This function calculates the (x,y) points to
    draw a polytropic curve of index j, for an ideall gas.
    
    Inputs:
    vi: float value for initial point volume
    pi: float value for initial point pressure
    j: polytopic index of the process
    
    Returns:
    (v_values, p_values) tuple containing:
    v_values: 1d numpy array of len=pts containing the x values of the points
    p_values: 1d numpy array of len=pts containing the y values of the points
    '''
    

    v_values = np.linspace(vi, vf, pts)
    p_values = np.empty((pts))
    for i in range(pts):
        p_values[i] = pi* (vi/v_values[i])**j
#        if p_values[i] > 2*p_max:                    # This sentence truncates high y_values to prevent memory overflow
#            p_values[i] = 2*p_max
            
    return v_values, p_values
    

In [4]:
def get_points_carnot(Ti,Tf,vi):
    pi = N*R*Ti/vi
    vf = (Ti/Tf * vi**(gamma-1.0))**(1/(gamma-1))
    pf = N*R*Tf/vf
    
    v_values, p_values = get_process(vi,pi,vf,gamma)
    
    curves.x = v_values
    curves.y = p_values

In [5]:
def get_isotherm(T):
    
    v_values = np.linspace(v_min, v_max, pts)
    p_values = np.empty((pts))
    
    for i in range(pts):
        p_values[i]  = N*R*T/v_values[i]
            
    return v_values, p_values
    

In [6]:
def update_points(change):
    vi = vi_slider.value
    Tc = Tc_slider.value
    Th = Th_slider.value
    
    pi = N*R*Tc/vi

    points.x = [vi]
    points.y = [pi]
    
    get_points_carnot(Tc, Th, vi)

In [7]:
def update_isotherms(change):
    Th = Th_slider.value
    Tc = Tc_slider.value
    
    x_values = np.empty((2,pts))
    y_values = np.empty((2,pts))
    
    v_values, p_values = get_isotherm(Th)
    x_values[0] = v_values
    y_values[0] = p_values
    
    v_values, p_values = get_isotherm(Tc)
    x_values[1] = v_values
    y_values[1] = p_values
    
    isotherms.x = x_values
    isotherms.y = y_values
    
    update_points(None)

In [8]:
#######################
###   PARAMETERS    ###
#######################

## Global Parameters

N = 1.0
R = 0.082057 # Ideal gas constant in atm*L/mol/K
C = 101.325 # Conversion factor from atm*L to J -> C = J/atmL
pts = 200 # Number of points for plotting each process
gamma = 5.0/3.0

## Limits of parameters (volumes in L, pressures in atm)

# Limits of the figure:
v_min = 0.01
v_max = 20.0
p_min = 0.01
p_max = 20.0

#Limits of the T sliders in the top block
Th_min = 300.0
Th_max = 400.0
Tc_min = 100.0
Tc_max = 200.0

# Gas values on left block
vi = 1.0
pi = 14.0
vf = 14.0
gamma = 5.0/3.0 # Adiabatic index (5/3 for monoatomic gases, 7/5 for diatomic gases at room temperature)
N = 1.0 # Number of mols of gas

########################
###CREATE THE FIGURES###
########################

fig_125_001 = bq.Figure(title='Prozesu politropikoak',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-right',
                legend_style= {'fill': 'white', 'stroke': 'grey'},
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                toolbar = True,
                layout=widgets.Layout(width='100%')
    )


scale_x = bqs.LinearScale(min = v_min, max = v_max)
scale_y = bqs.LinearScale(min = p_min, max = p_max)

axis_x = bqa.Axis(scale=scale_x,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(v_min, v_max, 11),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v (L)',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(
                scale=scale_y,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values= np.linspace(p_min, p_max, 6),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_125_001.axes = [axis_x,axis_y]

########################
####CREATE THE MARKS####
########################


curves = bqm.Lines(
                x = [], 
                y = [], 
                scales = {'x': scale_x, 'y': scale_y},
                display_legend=True
)

isotherms = bqm.Lines(
                x = [], 
                y = [], 
                scales = {'x': scale_x, 'y': scale_y},
                display_legend=True,
                labels = ["Foku beroa", "Foku hotza"],
                colors = ['#FF0000', '#FF7700'],
                opacities = [0.3,0.3]
)

points = bqm.Scatter(
    x = [],
    y = [],
    scales = {'x': scale_x, 'y': scale_y}, 
    opacities = [1.0],
    visible = True,
    colors = ['green'],
    labels=['Hasierako egoera'],
)      

fig_125_001.marks = [curves, isotherms, points]

########################
######  WIDGETS  #######
########################

Th_slider = widgets.FloatSlider(
    value=350.0,
    min=Th_min,
    max=Th_max,
    step=0.1,
    description='$T_H$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

Th_slider.observe(update_isotherms, 'value')

Tc_slider = widgets.FloatSlider(
    value=150.0,
    min=Tc_min,
    max=Tc_max,
    step=0.1,
    description='$T_C$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

Tc_slider.observe(update_isotherms, 'value')

process_dropdown = widgets.Dropdown(
    options=[('Monoatomikoa',5.0/3.0)],
    value=gamma,
    description='Zikloa',
    disabled=False,
    layout=widgets.Layout(width='100%')
)

#process_dropdown.observe(update_figure, 'value')

vi_slider = widgets.FloatSlider(
    value=10.0,
    min=v_min,
    max=v_max,
    step=0.1,
    description='$v_i$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

vi_slider.observe(update_points, 'value')

########################
######  LAYOUT  ########
########################


# Left Block ##
left_block_125_000 = widgets.VBox([], layout=widgets.Layout(width='20%', align_items='center'))
left_block_125_000.children = [Th_slider, Tc_slider, vi_slider
                               ]
                              
## Center Block ##
center_block_125_000 = widgets.VBox([], layout=widgets.Layout(width='80%', align_items='center'))
center_block_125_000.children = [fig_125_001]

## Main Block ##

main_block_123_000 = widgets.HBox([],layout=widgets.Layout(width='100%', align_items='center'))
main_block_123_000.children = [left_block_125_000, center_block_125_000]

In [9]:
def get_path(vi,pi,vf,pf):

    v_values = np.linspace(vi,vf,pts)
    
    if abs(vf-vi) < 0.01:  # Isochoric case
        p_values = np.linspace(pi,pf,pts)
    else:
        j = np.log(pi/pf)/np.log(vf/vi)
        p_values[i] = pi*(vi/v_values[i])**j

    return v_values, p_values

In [10]:
#carnot=[1.0,gamma,1.0,gamma]

In [11]:
#get_vertices(Tc,Th,v_min,v_max)
#vertices_v = np.empty((4))
#vertices_p = np.empty((4))

# Get first vertex:
#vertices_v[0] = vmin
#vertices_p[0] = N*R*Th/vmin

#Get third vertex:
#vertices_v[2] = v_max
#vertices_p[2] = N*R*Tc/vmax